# # Classifiers comparison on texts with naive Bayes assumption

In this session of laboratory we compare two models for categorical data probabilistic modeling: 
1. multivariate Bernoulli 
2. multinomial on a dataset 

We adopt a dataset on Twitter messages labelled with emotions (Joy vs Sadness).

The following program shows the loading of the data from a file.

Data are loaded into a matrix X adopting a sparse matrix representation, in order to save space and time.
Sparse matrix representation (in the csr format) represents in three "parallel" arrays the value of the matrix cells that are different from zero and the indices of those matrix cells.
The arrays are called: 
- data
- row
- col

- data[i] stores the value of the matrix cell #i whose indexes are contained in row[i] and col[i] 
- row[i] stores the index of the row in the matrix of the cell #i, 
- col[i] stores the index of the column of the cell #i.


The data file is in csv format.
Any Twitter message has been preprocessed by a Natural Language pipeline which eliminated stop words and substituted the interesting document elements with an integer identifier.  
The interesting document elements might be words, emoji or emoticons. The elements could be repeated in the same document and are uniquely identified in the documents by the same integer number (named "element_id" in the program). This "element_id" number will be used as the index of the column of the data matrix, for the purposes of storage of data.

Each row of the CSV file reports the content of a document (a Twitter message).It is formed as a list of integer number pairs, followed by a string which is the label of the document ("Joy" or "Sadness").
The first number of the pair is the identifier of a document element (the "element_id"); 
the second number of the pair is the count (frequency) of that element in that document.

The dataset has:

tot_n_docs (or rows in the file) =n_rows=11981

n_features (total number of distinct words in the corpus)=11288



The following program reads the data file and loads in a sparse way the matrix using the scipy.sparse library

In [ ]:

from numpy import ndarray, zeros
import numpy as np
import scipy
from scipy.sparse import csr_matrix

class_labels = ["Joy","Sadness"]
n_features=11288 # number of columns in the matrix = number of features (distinct elements in the documents)
n_rows=11981 # number rows of the matrix
n_elements=71474 # number of the existing values in the matrix (not empty, to be loaded in the matrix in a sparse way)

#path_training="/Users/meo/Documents/Didattica/Laboratorio-15-16-Jupyter/"
path_training="./"
file_name="joy_sadness6000.txt"

# declare the row and col arrays with the indexes of the matrix cells (non empty) to be loaded from file
# they are needed because the matrix is sparse and we load in the matrix only the elements which are present
row=np.empty(n_elements, dtype=int)
col=np.empty(n_elements, dtype=int)
data=np.empty(n_elements, dtype=int)

row_n=0 # number of current row to be read and managed
cur_el=0 # position in the three arrays: row, col and data
twitter_labels=[] # list of class labels (target array) of the documents (twitter) that will be read from the input file
twitter_target=[] # list of 0/1 for class labels
with open(path_training + file_name, "r") as fi:
    for line in fi:
        el_list=line.split(',')  # list of integers read from a row of the file
        l=len(el_list)
        last_el=el_list[l-1] # I grab the last element in the list which is the class label
        class_name=last_el.strip() # eliminate the '\n'
        twitter_labels.append(class_name)
        # twitter_labels contains the labels (Joy/Sadness); twitter_target contains 0/1 for the respective labels
        if (class_name==class_labels[0]):
           twitter_target.append(0)
        else:
           twitter_target.append(1)
        i=0 # I start reading all the doc elements from the beginning of the list
        while(i<(l-1)):
            element_id=int(el_list[i]) # identifier of the element in the document equivalent to the column index
            element_id=element_id-1 # the index starts from 0 (the read id starts from 1)
            i=i+1
            value_cell=int(el_list[i]) # make access to the following value in the file which is the count of the element in the documento 
            i=i+1
            row[cur_el]=row_n # load the data in the three arrays: the first two are the row and col indexes; the last one is the matrix cell value
            col[cur_el]=element_id
            data[cur_el]=value_cell
            cur_el=cur_el+1
        row_n=row_n+1
fi.close
print("final n_row="+str(row))
# loads the matrix by means of the indexes and the values in the three arrays just filled
twitter_data=csr_matrix((data, (row, col)), shape=(n_rows, n_features)).toarray()
print("resulting matrix:")
print(twitter_data)
print(twitter_labels)
print(twitter_target)


Write a program in the following cell that splits the data matrix in training and test set (by random selection) and predicts the class (Joy/Sadness) of the messages on the basis of the words. 
Consider the two possible models:
multivariate Bernoulli and multinomial Bernoulli.
Find the accuracy of the models and test is the observed differences are significant.

In [4]:
from sklearn.model_selection import train_test_split, cross_val_score

In [5]:
X_train, X_test, y_train, y_test = train_test_split(twitter_data, twitter_labels, test_size=0.2, random_state=0) #prendiamo il 20% come test set e mettiamo il seed (random state) a 0
X_train.shape, X_test.shape

((9584, 11288), (2397, 11288))

##Partiamo con Bernulli Multivariato

In [6]:
from sklearn.naive_bayes import BernoulliNB

clf = BernoulliNB()
clf.fit(X_train, y_train)
print(clf.score(X_test, y_test))
scores_multivariate = cross_val_score(clf, twitter_data, twitter_labels, cv=5)
scores_multivariate

0.9578639966624948


array([0.95953275, 0.95075125, 0.94824708, 0.95033389, 0.9490818 ])

Abbiamo un'accuratezza del 0.95 sul test set

##Ora proviamo con Bernulli Multinomial

In [9]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB()
clf.fit(X_train, y_train)
print(clf.score(X_test, y_test))
scores_multinomial = cross_val_score(clf, twitter_data, twitter_labels, cv=5)
scores_multinomial

0.9561952440550688


array([0.95661243, 0.95075125, 0.94323873, 0.94782972, 0.94866444])

Otteniamo praticamente la stessa accuratezza, ovviamente abbiamo usato la CV per verificare meglio le differenze ma in sostanza nulla di diverso a livello di score

#Parte aggiuntiva: Experiments design and statistical tests on results

###Estendete l'esercizio n.4 sullo Jupyter Notebook aggiungendo al punto 2 un test statistico per determinare se le differenze osservate siano statisticamente significative tra i classificatori probabilistici sui testi di tipo Multivariato e Multinomiale.

In [14]:
from scipy.stats import ttest_ind
t_value, p_value = ttest_ind(scores_multivariate, scores_multinomial)
print("t_value")
print(t_value)
print("\np_value")
print(p_value)


t_value
0.7281234299852103

p_value
0.487300250839351


Il __p_value__ è una probabilità quantifica la probabilità di osservare dei valori assumento l'ipotesi nulla, in sostanza ci dice quanto gli score ottenuti dalle due distribuzioni sono correlate.

Il p_value ci dice poco in quanto sono esperimenti indipendenti, quindi guardiamo il t_value

Il __t_value__ è rapporto tra la differenza delle due performance e la differenza tra le performance all'interno degli esempi.
Maggiore è il valore di t più saranno diversi gli elementi delle distribuzioni, se avessimo per esempoi un t_value = 5 vuol dire che gli elementi dei due gruppi sono 5 volte diversi tra di loro

Possiamo osservare che il t-value è basso e pertanto non ci sono differenze sostanziali in termini di performance tra il modello multinomiale e quello multivariato. Ciò lo possiamo osservare considerando anche l'accuratezza media e la deviazione standard ottenuta dai due modelli

In [ ]:
np.mean(scores_multinomial)*100, np.std(scores_multinomial)*100, np.mean(scores_multivariate)*100, np.std(scores_multivariate)*100

(94.94193144881295, 0.43545669579245355, 95.15893545284418, 0.4070251866845663)

---